In [1]:
! pip install datasets transformers

     |████████████████████████████████| 264 kB 7.1 MB/s 
     |████████████████████████████████| 2.8 MB 43.2 MB/s 
     |████████████████████████████████| 119 kB 62.2 MB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 243 kB 56.9 MB/s 
     |████████████████████████████████| 895 kB 53.9 MB/s 
     |████████████████████████████████| 636 kB 50.5 MB/s 
     |████████████████████████████████| 3.3 MB 43.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import datasets
dataset_train = datasets.load_dataset("discovery","discovery",split=datasets.ReadInstruction('train', to=100, unit='%'))
dataset_test = datasets.load_dataset("discovery","discovery",split=datasets.ReadInstruction('test', to=100, unit='%'))
dataset_valid = datasets.load_dataset("discovery","discovery",split=datasets.ReadInstruction('validation', to=100, unit='%'))

Downloading:   0%|          | 0.00/3.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/146M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset discovery downloaded and prepared to /root/.cache/huggingface/datasets/discovery/discovery/1.0.0/542fab7a9ddc1d9726160355f7baa06a1ccc44c40bc8e12c09e9bc743aca43a2. Subsequent calls will reuse this data.


Reusing dataset discovery (/root/.cache/huggingface/datasets/discovery/discovery/1.0.0/542fab7a9ddc1d9726160355f7baa06a1ccc44c40bc8e12c09e9bc743aca43a2)
Reusing dataset discovery (/root/.cache/huggingface/datasets/discovery/discovery/1.0.0/542fab7a9ddc1d9726160355f7baa06a1ccc44c40bc8e12c09e9bc743aca43a2)


In [4]:
task = "discovery"
#model_checkpoint = "xlm-roberta-base"
model_checkpoint = "/content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/"
batch_size = 16

In [5]:
from datasets import load_dataset, load_metric

In [6]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", use_fast=True)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [8]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [0, 35378, 4, 903, 1632, 149357, 38, 2, 2, 3493, 903, 149357, 60899, 678, 442, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:


task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
    "discovery":("sentence1", "sentence2")
}

sentence1_key, sentence2_key = task_to_keys['discovery']

In [10]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [11]:
sentence2_key

'sentence2'

In [12]:
encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_test = dataset_test.map(preprocess_function, batched=True)
encoded_dataset_valid = dataset_valid.map(preprocess_function, batched=True)

  0%|          | 0/1566 [00:00<?, ?ba/s]

  0%|          | 0/87 [00:00<?, ?ba/s]

  0%|          | 0/87 [00:00<?, ?ba/s]

In [13]:
num_labels = dataset_train.features['label'].num_classes

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [15]:
model_checkpoint

'/content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/'

In [30]:
metric_name = "accuracy"

In [16]:
args = TrainingArguments(
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    output_dir="/content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval"
)

In [32]:
actual_task = "mnli" if task == "discovery" else task
#dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [ ]:
metric

In [17]:
import numpy as np
def compute_metrics(eval_pred):
    y_pred, y_true = eval_pred
    y_pred = np.argmax(y_pred, axis=1)
    correct_predictions = 0
    
    for yt, yp in zip(y_true, y_pred):    
        if yt == yp:
            correct_predictions += 1
    
    #returns accuracy
    return correct_predictions / len(y_true)

In [18]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_valid,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running training *****
  Num examples = 1566000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 146814


Epoch,Training Loss,Validation Loss
1,2.956200,3.090380


The following columns in the evaluation set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Evaluation *****
  Num examples = 87000
  Batch size = 16


Saving model checkpoint to /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938
Configuration saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/config.json
Model weights saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/eval/checkpoint-48938/special_tokens_map.json


Epoch,Training Loss,Validation Loss
1,2.956200,3.090380


In [19]:
predictions = trainer.predict(encoded_dataset_valid)

The following columns in the test set  don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx.
***** Running Prediction *****
  Num examples = 87000
  Batch size = 16


In [ ]:
trainer.save_model("/content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/")

Saving model checkpoint to /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/
Configuration saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/config.json
Model weights saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/DiscourseMarkers/Discovery/xlm-roberta-base/special_tokens_map.json


In [41]:
predictions

PredictionOutput(predictions=array([[ 0.8006021 ,  0.4520864 , -2.3425896 , ...,  2.6190138 ,
         0.10056576,  1.4761968 ],
       [ 1.3371739 , -0.7145302 , -1.869166  , ..., -1.1494094 ,
         0.6596953 ,  0.10003195],
       [ 0.7194921 , -2.768294  , -1.5274578 , ..., -0.8449462 ,
         0.44125813, -0.9899065 ],
       ...,
       [ 0.15691677, -1.5138341 , -1.3289758 , ..., -1.6161181 ,
         0.7399995 ,  4.079126  ],
       [ 0.54617906, -0.2712015 , -2.0470543 , ...,  2.4358234 ,
         1.6147344 ,  0.7490102 ],
       [ 0.44081774, -1.3292745 , -0.27483478, ..., -2.7263393 ,
        -1.3362863 , -0.29662368]], dtype=float32), label_ids=array([ 43, 108,  86, ..., 169,   5,  51]), metrics={'test_loss': 3.198350191116333, 'test_runtime': 256.7304, 'test_samples_per_second': 338.877, 'test_steps_per_second': 21.182})

In [20]:
compute_metrics((predictions.predictions,predictions.label_ids))

0.2650344827586207

In [ ]:
dataset_valid["label"]

In [ ]:
predictions.predictions

array([[ 0.03785788,  0.31931916, -0.92735624, ...,  2.0858865 ,
         0.10702702,  0.7280692 ],
       [ 0.16956301, -0.72486204, -0.26084444, ..., -0.27959526,
        -0.20676509, -0.57695544],
       [ 0.1990494 , -0.6890304 , -1.1390895 , ...,  0.41601655,
         0.32574737,  0.06615347],
       ...,
       [-0.12880795, -0.53302413,  1.5652626 , ..., -0.5106398 ,
        -1.474261  ,  0.59660226],
       [ 0.11063018, -1.421752  ,  1.8063176 , ..., -1.0156395 ,
        -1.7418902 , -0.17828219],
       [ 0.4896539 ,  0.6738566 , -0.98236465, ...,  0.27730873,
         0.2279788 ,  0.68641776]], dtype=float32)

In [ ]:
y_pred = np.argmax(predictions.predictions,axis=1)
y_label = predictions.label_ids

In [ ]:
df_results = pd.DataFrame(data={"y_pred":y_pred, "y_label":y_label})

In [ ]:
df_results[df_results.y_pred==df_results.y_label].head()

,y_pred,y_label
1,108,108
9,94,94
19,43,43
47,43,43
48,59,59


In [ ]:
!pip install shap

     |████████████████████████████████| 356 kB 8.4 MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491635 sha256=6ce250536ef960fbc54d0c5b44ca04c974e5207ed25bc9101bfc482bd68c2eea
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built shap


In [ ]:
import shap
# define a prediction function
def f(x):
    #tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=500, truncation=True) for v in x]).cuda()
    tv = torch.tensor([tokenizer.encode(v[sentence1_key], v[sentence2_key], truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [ ]:
dataset_valid

Dataset({
    features: ['sentence1', 'sentence2', 'label', 'idx'],
    num_rows: 17400
})

In [ ]:
model(torch.tensor([tokenizer.encode(v[sentence1_key], v[sentence2_key], padding='max_length', max_length=32, truncation=True) for v in dataset_valid]).cuda())

RuntimeError: ignored

In [ ]:
dataset_train[0]['sentence2']

'Sanchez became involved with the American GI Forum, the National Council of La Raza and various migrant farm worker organizations.'

In [ ]:
# explain the model's predictions on IMDB reviews
import torch
import scipy as sp
imdb_train = encoded_dataset_valid
shap_values = explainer(dataset_train[:10],  fixed_context=1)

['<mask><mask>']


TypeError: ignored

In [ ]:
sentence2_key

'sentence2'

In [ ]:
shap_values

.values =
array([array([ 0.29552615, -0.21946466, -0.28856218,  0.29550946]),
       array([ 0.05712724,  0.49579358, -0.05123377,  0.05707264]),
       array([ 4.45048014e-06, -1.04268964e+00,  2.63849894e-05]),
       array([ 0.26203942, -0.19474769, -0.13381696,  0.26207662])],
      dtype=object)

.base_values =
array([-4.80142403, -4.80142403, -4.1008873 , -4.80142403])

.data =
array([list(['', 'sentence', '1', '']), list(['', 'sentence', '2', '']),
       list(['', 'label', '']), list(['', 'id', 'x', ''])], dtype=object)

In [ ]:
# plot the first sentence's explanation
shap.plots.text(shap_values[3])